In [1]:
import torch
import torch.nn as nn
import torch.fft as fft
from torch.utils.data import Dataset

import torchvision
from torchvision.transforms import v2
from torchvision import models
import matplotlib.pyplot as plt

In [2]:
import os
import math
import numpy
from tqdm import tqdm
import cv2
import lmdb
import pickle

In [ ]:
def load_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    transform = v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((128, 128)),
    ])
    img = transform(img)
    return img

In [ ]:
path = os.path.join('image.jpeg')
image = load_image(path)
fft_image = fft.fft2(image, dim=(-2,-1))
fft_shift = fft.fftshift(fft_image, dim=(-2,-1))
magnitude = torch.log(torch.abs(fft_shift)+ 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes = axes.flatten()

axes[0].set_title('Original')
axes[0].imshow(image.permute(1,2,0), cmap='gray')
axes[0].axis('off')

axes[1].set_title('Magnitude Spectrum')
axes[1].imshow(magnitude.permute(1,2,0), cmap='gray')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('fft_img.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
rows, cols = image.squeeze(0).shape
crow, ccol = rows//2, cols//2
radius = 20

mask = torch.zeros((rows, cols))
y, x = torch.meshgrid(torch.arange(0, rows), torch.arange(0, cols), indexing='ij')
mask_area = torch.sqrt((x - ccol)**2 + (y - crow)**2)

mask_h = (mask_area > radius).float()
mask_l = (mask_area <= radius).float()

In [ ]:
high_filtered_fft = fft_shift * mask_h
high_filtered_image = torch.abs(fft.ifft2(fft.ifftshift(high_filtered_fft)))
magnitude_high_filtered_image = torch.log(torch.abs(high_filtered_fft)+ 1)

low_filtered_fft = fft_shift * mask_l
low_filtered_image = torch.abs(fft.ifft2(fft.ifftshift(low_filtered_fft)))
magnitude_low_filtered_image = torch.log(torch.abs(low_filtered_fft)+ 1)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 5))
axes = axes.flatten()

axes[0].set_title('Original')
axes[0].imshow(image.permute(1,2,0), cmap='gray')
axes[0].axis('off')

axes[1].set_title('FFT')
axes[1].imshow(magnitude.permute(1,2,0), cmap='gray')
axes[1].axis('off')

axes[2].set_title('Low Pass Filter')
axes[2].imshow(magnitude_low_filtered_image.permute(1,2,0), cmap='gray')
axes[2].axis('off')

axes[3].set_title('High Pass Filter')
axes[3].imshow(magnitude_high_filtered_image.permute(1,2,0), cmap='gray')
axes[3].axis('off')

axes[4].set_title('Low Filter Image')
axes[4].imshow(low_filtered_image.permute(1,2,0), cmap='gray')
axes[4].axis('off')

axes[5].set_title('High Filter Image')
axes[5].imshow(high_filtered_image.permute(1,2,0), cmap='gray')
axes[5].axis('off')

axes[6].set_title('Low Filter Mask')
axes[6].imshow(mask_l, cmap='gray')
axes[6].axis('off')

axes[7].set_title('High Filter Mask')
axes[7].imshow(mask_h, cmap='gray')
axes[7].axis('off')

plt.savefig('filter_img.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 4, figsize=(15, 10))
axes = axes.flatten()

# Original Image
axes[0].set_title('Original Image')
axes[0].imshow(image.squeeze(), cmap='gray')
axes[0].axis('off')

# FFT Spectrum
axes[1].set_title('FFT Magnitude Spectrum')
axes[1].imshow(magnitude.squeeze(), cmap='gray')
axes[1].axis('off')

# Low-Pass Filtered Spectrum
axes[2].set_title('Low-Pass Filter Spectrum')
axes[2].imshow(magnitude_low_filtered.squeeze(), cmap='gray')
axes[2].axis('off')

# High-Pass Filtered Spectrum
axes[3].set_title('High-Pass Filter Spectrum')
axes[3].imshow(magnitude_high_filtered.squeeze(), cmap='gray')
axes[3].axis('off')

# Inverse FFT (Reconstructed) Images
axes[4].set_title('Reconstructed Low-Pass Image')
axes[4].imshow(low_filtered_image.squeeze(), cmap='gray')
axes[4].axis('off')

axes[5].set_title('Reconstructed High-Pass Image')
axes[5].imshow(high_filtered_image.squeeze(), cmap='gray')
axes[5].axis('off')

# Show Filters
axes[6].set_title('Low-Pass Mask')
axes[6].imshow(mask_l, cmap='gray')
axes[6].axis('off')

axes[7].set_title('High-Pass Mask')
axes[7].imshow(mask_h, cmap='gray')
axes[7].axis('off')

In [ ]:
model = models.googlenet(weights='GoogLeNet_Weights.DEFAULT')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
def change_layer(layer):
    fft_conv = FFTConvNet(layer, 'low')
    return fft_conv

In [ ]:
# Check the output channels of the original Conv2d layer
original_conv = model.inception3a.branch2[0].conv
print("Original Conv2d output channels:", original_conv.out_channels)

# Replace the Conv2d layer with FFTConvNet
fft_conv = change_layer(original_conv).to(device)
model.inception3a.branch2[0].conv = fft_conv

# Verify the output channels of the FFTConvNet layer
print("FFTConvNet output channels:", fft_conv.conv_layer.out_channels)

In [ ]:
model = models.alexnet(weights=None)

In [ ]:
model.classifier

In [ ]:
model.features = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(9216, out_features=1024),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=1024, out_features=512),
    nn.LeakyReLU(),
    nn.Linear(in_features=512, out_features=3)
)

In [ ]:
model

In [ ]:
dummy_input = torch.randn(1, 3, 227, 227)
outputs = model(dummy_input)
print("Outputs: ", outputs)

In [ ]:
model = models.alexnet(weights=)